# Stable Baselines - Hindsight Experience Replay on Highway Env

Github Repo: [https://github.com/DLR-RM/stable-baselines3](https://github.com/DLR-RM/stable-baselines3)

Highway env: [https://github.com/eleurent/highway-env](https://github.com/eleurent/highway-env)

[RL Baselines3 Zoo](https://github.com/DLR-RM/rl-baselines3-zoo) is a training framework for Reinforcement Learning (RL), using Stable Baselines3.

It provides scripts for training, evaluating agents, tuning hyperparameters, plotting results and recording videos.

Documentation is available online: [https://stable-baselines3.readthedocs.io/](https://stable-baselines3.readthedocs.io/)

## Install Dependencies and Stable Baselines Using Pip


```
pip install stable-baselines3[extra]
```

In [1]:
# for autoformatting
# %load_ext jupyter_black

In [2]:
# Install stable-baselines latest version
!pip install "stable-baselines3[extra]>=2.0.0a4"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.2/182.2 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.9/953.9 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.7/434.7 kB 5.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached Farama_Notifications-0.0.4-py3-none-any.whl (2.5 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 7.2 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2

In [3]:
# Install highway-env
!pip install highway-env

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.0/104.0 kB 1.2 MB/s eta 0:00:00


## Import policy, RL agent, ...

In [4]:
import gymnasium as gym
import highway_env
import numpy as np

from stable_baselines3 import HerReplayBuffer, SAC, DDPG
from stable_baselines3.common.noise import NormalActionNoise

## Create the Gym env and instantiate the agent

For this example, we will be using the parking environment from the [highway-env](https://github.com/Farama-Foundation/HighwayEnv) repo by @eleurent.

The parking env is a goal-conditioned continuous control task, in which the vehicle must park in a given space with the appropriate heading.


![parking-env](https://raw.githubusercontent.com/eleurent/highway-env/gh-media/docs/media/parking-env.gif)



### Train Soft Actor-Critic (SAC) agent

Here, we use HER "future" goal sampling strategy, where we create 4 artificial transitions per real transition

Note: the hyperparameters (network architecture, discount factor, ...) were tuned for this task

In [17]:
env = gym.make("parking-v0")

In [6]:
# SAC hyperparams:
model = SAC(
    "MultiInputPolicy",
    env,
    replay_buffer_class=HerReplayBuffer,
    replay_buffer_kwargs=dict(
        n_sampled_goal=4,
        goal_selection_strategy="future",
    ),
    verbose=1,
    buffer_size=int(1e6),
    learning_rate=1e-3,
    gamma=0.95,
    batch_size=256,
    policy_kwargs=dict(net_arch=[256, 256, 256]),
)

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [7]:
import warnings
# Suppress warnings in Colab
warnings.filterwarnings("ignore")

# Train for 1e5 steps
model.learn(int(1e5))
# Save the trained agent
model.save('her_sac_highway')

Streaming output truncated to the last 5000 lines.
|    success_rate    | 0.94     |
| time/              |          |
|    episodes        | 2136     |
|    fps             | 38       |
|    time_elapsed    | 2012     |
|    total_timesteps | 76976    |
| train/             |          |
|    actor_loss      | 1.59     |
|    critic_loss     | 0.0129   |
|    ent_coef        | 0.00503  |
|    ent_coef_loss   | -0.181   |
|    learning_rate   | 0.001    |
|    n_updates       | 76875    |
---------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 23.1     |
|    ep_rew_mean     | -7.47    |
|    success_rate    | 0.94     |
| time/              |          |
|    episodes        | 2140     |
|    fps             | 38       |
|    time_elapsed    | 2014     |
|    total_timesteps | 77071    |
| train/             |          |
|    actor_loss      | 1.64     |
|    critic_loss     | 0.0228   |
|    ent_coef        | 0.00492 

In [8]:
# Load saved model
model = SAC.load('her_sac_highway', env=env)

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


#### Evaluate the agent

In [14]:
!pip install git+https://github.com/DLR-RM/stable-baselines3

  Cloning https://github.com/DLR-RM/stable-baselines3 to /tmp/pip-req-build-hmrq9ex6
  Running command git clone --filter=blob:none --quiet https://github.com/DLR-RM/stable-baselines3 /tmp/pip-req-build-hmrq9ex6
  Resolved https://github.com/DLR-RM/stable-baselines3 to commit 4af4a32d1b5acb06d585ef7bb0a00c83810fe5c3
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [16]:
import sys
from tqdm.notebook import trange
!pip install tensorboardx gym pyvirtualdisplay
!apt-get install -y xvfb ffmpeg
!git clone https://github.com/Farama-Foundation/HighwayEnv.git 2> /dev/null
sys.path.insert(0, '/content/HighwayEnv/scripts/')
from utils import record_videos, show_videos

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
xvfb is already the newest version (2:21.1.4-2ubuntu1.7~22.04.10).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.


In [11]:
model = SAC.load('her_sac_highway', env=env)
env = gym.make("parking-v0", render_mode='rgb_array')
env = record_videos(env)
for _ in range(500):
    obs, info = env.reset()
    done = truncated = False
    while not (done or truncated):
        action, _ = model.predict(obs)
        obs, reward, done, truncated, info = env.step(action)
        env.render()

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Moviepy - Building video /content/videos/rl-video-episode-0.mp4.
Moviepy - Writing video /content/videos/rl-video-episode-0.mp4



Moviepy - Done !
Moviepy - video ready /content/videos/rl-video-episode-0.mp4
Moviepy - Building video /content/videos/rl-video-episode-1.mp4.
Moviepy - Writing video /content/videos/rl-video-episode-1.mp4



Moviepy - Done !
Moviepy - video ready /content/videos/rl-video-episode-1.mp4
Moviepy - Building video /content/videos/rl-video-episode-2.mp4.
Moviepy - Writing video /content/videos/rl-video-episode-2.mp4



Moviepy - Done !
Moviepy - video ready /content/videos/rl-video-episode-2.mp4
Moviepy - Building video /content/videos/rl-video-episode-3.mp4.
Moviepy - Writing video /content/videos/rl-video-episode-3.mp4



Moviepy - Done !
Moviepy - video ready /content/videos/rl-video-episode-3.mp4
Moviepy - Building video /content/videos/rl-video-episode-4.mp4.
Moviepy - Writing video /content/videos/rl-video-episode-4.mp4



...

Moviepy - Done !
Moviepy - video ready /content/videos/rl-video-episode-495.mp4
Moviepy - Building video /content/videos/rl-video-episode-496.mp4.
Moviepy - Writing video /content/videos/rl-video-episode-496.mp4



Moviepy - Done !
Moviepy - video ready /content/videos/rl-video-episode-496.mp4
Moviepy - Building video /content/videos/rl-video-episode-497.mp4.
Moviepy - Writing video /content/videos/rl-video-episode-497.mp4



Moviepy - Done !
Moviepy - video ready /content/videos/rl-video-episode-497.mp4
Moviepy - Building video /content/videos/rl-video-episode-498.mp4.
Moviepy - Writing video /content/videos/rl-video-episode-498.mp4



Moviepy - Done !
Moviepy - video ready /content/videos/rl-video-episode-498.mp4
Moviepy - Building video /content/videos/rl-video-episode-499.mp4.
Moviepy - Writing video /content/videos/rl-video-episode-499.mp4



Moviepy - Done !
Moviepy - video ready /content/videos/rl-video-episode-499.mp4


In [12]:
!zip -r /content/videos.zip /content/videos

  adding: content/videos/ (stored 0%)
  adding: content/videos/rl-video-episode-35.meta.json (deflated 8%)
  adding: content/videos/rl-video-episode-46.meta.json (deflated 11%)
  adding: content/videos/rl-video-episode-61.mp4 (deflated 6%)
  adding: content/videos/rl-video-episode-200.mp4 (deflated 5%)
  adding: content/videos/rl-video-episode-57.meta.json (deflated 8%)
  adding: content/videos/rl-video-episode-12.mp4 (deflated 5%)
  adding: content/videos/rl-video-episode-327.meta.json (deflated 8%)
  adding: content/videos/rl-video-episode-208.mp4 (deflated 5%)
  adding: content/videos/rl-video-episode-458.meta.json (deflated 9%)
  adding: content/videos/rl-video-episode-364.mp4 (deflated 6%)
  adding: content/videos/rl-video-episode-139.mp4 (deflated 5%)
  adding: content/videos/rl-video-episode-472.meta.json (deflated 8%)
  adding: content/videos/rl-video-episode-161.mp4 (deflated 6%)
  adding: content/videos/rl-video-episode-297.mp4 (deflated 5%)
  adding: content/videos/rl-video-

In [13]:
# we use the gym >v.26 API here. Note that you could also wrap the env in a DummyVecEnv
# which allows you to use a simplified API
obs, _ = env.reset()

# Evaluate the agent
episode_reward = 0
for _ in range(1000):
    action, _ = model.predict(obs, deterministic=True)
    obs, reward, terminated, truncated, info = env.step(action)
    done = truncated or terminated
    episode_reward += reward
    if done or info.get("is_success", False):
        print("Reward:", episode_reward, "Success?", info.get("is_success", False))
        episode_reward = 0.0
        obs, _ = env.reset()

Moviepy - Building video /content/videos/rl-video-episode-500.mp4.
Moviepy - Writing video /content/videos/rl-video-episode-500.mp4



Moviepy - Done !
Moviepy - video ready /content/videos/rl-video-episode-500.mp4
Reward: -6.152391009034989 Success? True
Moviepy - Building video /content/videos/rl-video-episode-501.mp4.
Moviepy - Writing video /content/videos/rl-video-episode-501.mp4



Moviepy - Done !
Moviepy - video ready /content/videos/rl-video-episode-501.mp4
Reward: -5.035450825329911 Success? True
Moviepy - Building video /content/videos/rl-video-episode-502.mp4.
Moviepy - Writing video /content/videos/rl-video-episode-502.mp4



Moviepy - Done !
Moviepy - video ready /content/videos/rl-video-episode-502.mp4
Reward: -8.429987716271828 Success? True
Moviepy - Building video /content/videos/rl-video-episode-503.mp4.
Moviepy - Writing video /content/videos/rl-video-episode-503.mp4



Moviepy - Done !
Moviepy - video ready /content/videos/rl-video-episode-503.mp4
Reward: -7.544497513973789 Success? True
Moviepy - Building video /content/videos/rl-video-episode-504.mp4.
Moviepy - Writing video /content/videos/rl-video-episode-504.mp4



...

Moviepy - Done !
Moviepy - video ready /content/videos/rl-video-episode-545.mp4
Reward: -4.957568540702943 Success? True
Moviepy - Building video /content/videos/rl-video-episode-546.mp4.
Moviepy - Writing video /content/videos/rl-video-episode-546.mp4



Moviepy - Done !
Moviepy - video ready /content/videos/rl-video-episode-546.mp4
Reward: -7.153087290113137 Success? True
Moviepy - Building video /content/videos/rl-video-episode-547.mp4.
Moviepy - Writing video /content/videos/rl-video-episode-547.mp4



Moviepy - Done !
Moviepy - video ready /content/videos/rl-video-episode-547.mp4
Reward: -3.4370706610483417 Success? True
Moviepy - Building video /content/videos/rl-video-episode-548.mp4.
Moviepy - Writing video /content/videos/rl-video-episode-548.mp4



Moviepy - Done !
Moviepy - video ready /content/videos/rl-video-episode-548.mp4
Reward: -9.003869297304337 Success? True
Moviepy - Building video /content/videos/rl-video-episode-549.mp4.
Moviepy - Writing video /content/videos/rl-video-episode-549.mp4



Moviepy - Done !
Moviepy - video ready /content/videos/rl-video-episode-549.mp4
Reward: -4.509464595493106 Success? True


### Train DDPG agent

In [5]:
env = gym.make("parking-v0")

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [6]:
# Create the action noise object that will be used for exploration
n_actions = env.action_space.shape[0]
noise_std = 0.2
action_noise = NormalActionNoise(
    mean=np.zeros(n_actions), sigma=noise_std * np.ones(n_actions)
)

model = DDPG(
    "MultiInputPolicy",
    env,
    replay_buffer_class=HerReplayBuffer,
    replay_buffer_kwargs=dict(
        n_sampled_goal=4,
        goal_selection_strategy="future",
    ),
    verbose=1,
    buffer_size=int(1e6),
    learning_rate=1e-3,
    action_noise=action_noise,
    gamma=0.95,
    batch_size=256,
    policy_kwargs=dict(net_arch=[256, 256, 256]),
)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [20]:
import warnings
# Suppress warnings in Colab
warnings.filterwarnings("ignore")

# Train for 2e5 steps
model.learn(int(2e5))
# Save the trained agent
model.save('her_ddpg_highway')

Streaming output truncated to the last 5000 lines.
|    time_elapsed    | 3806     |
|    total_timesteps | 172988   |
| train/             |          |
|    actor_loss      | 0.619    |
|    critic_loss     | 0.00787  |
|    learning_rate   | 0.001    |
|    n_updates       | 172887   |
---------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 20.7     |
|    ep_rew_mean     | -7.48    |
|    success_rate    | 0.86     |
| time/              |          |
|    episodes        | 5928     |
|    fps             | 45       |
|    time_elapsed    | 3808     |
|    total_timesteps | 173081   |
| train/             |          |
|    actor_loss      | 0.559    |
|    critic_loss     | 0.0103   |
|    learning_rate   | 0.001    |
|    n_updates       | 172980   |
---------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 20.7     |
|    ep_rew_mean     | -7.49   

In [9]:
# Load saved model
model = DDPG.load('her_ddpg_highway', env=env)

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


#### Evaluate the agent

In [10]:
# we use the gym >v.26 API here. Note that you could also wrap the env in a DummyVecEnv
# which allows you to use the old gym API a simplified API
obs, _ = env.reset()

# Evaluate the agent
episode_reward = 0
for _ in range(1000):
    action, _ = model.predict(obs, deterministic=True)
    obs, reward, terminated, truncated, info = env.step(action)
    done = truncated or terminated
    episode_reward += reward
    if done or info.get("is_success", False):
        print("Reward:", episode_reward, "Success?", info.get("is_success", False))
        episode_reward = 0.0
        obs, _ = env.reset()

Reward: -6.475268265640113 Success? True
Reward: -9.976535206166425 Success? True
Reward: -8.984516334853577 Success? True
Reward: -5.499656405419206 Success? True
Reward: -7.314162136307797 Success? True
Reward: -7.249408275737432 Success? True
Reward: -8.097351898357674 Success? True
Reward: -7.269286754066262 Success? True
Reward: -7.635319959955108 Success? True
Reward: -7.918705932087883 Success? True
Reward: -6.756679988704863 Success? True
Reward: -9.829774577860972 Success? True
Reward: -9.109460870519259 Success? True
Reward: -7.257721577319546 Success? True
Reward: -6.504470115007772 Success? True
Reward: -5.774842286718739 Success? True
Reward: -10.263574211249807 Success? True
Reward: -9.289851199929808 Success? True
Reward: -5.459978534618745 Success? True
Reward: -8.193102826716196 Success? True
Reward: -8.708914411636385 Success? True
Reward: -7.241971857516917 Success? True
Reward: -2.7858467565355167 Success? True
Reward: -5.495357573193169 Success? True
Reward: -7.398

In [17]:
model = DDPG.load('her_ddpg_highway', env=env)
env = gym.make("parking-v0", render_mode='rgb_array')
env = record_videos(env)
for _ in range(500):
    obs, info = env.reset()
    done = truncated = False
    while not (done or truncated):
        action, _ = model.predict(obs)
        obs, reward, done, truncated, info = env.step(action)
        env.render()

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Moviepy - Building video /content/videos/rl-video-episode-0.mp4.
Moviepy - Writing video /content/videos/rl-video-episode-0.mp4



Moviepy - Done !
Moviepy - video ready /content/videos/rl-video-episode-0.mp4
Moviepy - Building video /content/videos/rl-video-episode-1.mp4.
Moviepy - Writing video /content/videos/rl-video-episode-1.mp4



Moviepy - Done !
Moviepy - video ready /content/videos/rl-video-episode-1.mp4
Moviepy - Building video /content/videos/rl-video-episode-2.mp4.
Moviepy - Writing video /content/videos/rl-video-episode-2.mp4



Moviepy - Done !
Moviepy - video ready /content/videos/rl-video-episode-2.mp4
Moviepy - Building video /content/videos/rl-video-episode-3.mp4.
Moviepy - Writing video /content/videos/rl-video-episode-3.mp4



Moviepy - Done !
Moviepy - video ready /content/videos/rl-video-episode-3.mp4
Moviepy - Building video /content/videos/rl-video-episode-4.mp4.
Moviepy - Writing video /content/videos/rl-video-episode-4.mp4



...

Moviepy - Done !
Moviepy - video ready /content/videos/rl-video-episode-495.mp4
Moviepy - Building video /content/videos/rl-video-episode-496.mp4.
Moviepy - Writing video /content/videos/rl-video-episode-496.mp4



Moviepy - Done !
Moviepy - video ready /content/videos/rl-video-episode-496.mp4
Moviepy - Building video /content/videos/rl-video-episode-497.mp4.
Moviepy - Writing video /content/videos/rl-video-episode-497.mp4



Moviepy - Done !
Moviepy - video ready /content/videos/rl-video-episode-497.mp4
Moviepy - Building video /content/videos/rl-video-episode-498.mp4.
Moviepy - Writing video /content/videos/rl-video-episode-498.mp4



Moviepy - Done !
Moviepy - video ready /content/videos/rl-video-episode-498.mp4
Moviepy - Building video /content/videos/rl-video-episode-499.mp4.
Moviepy - Writing video /content/videos/rl-video-episode-499.mp4



Moviepy - Done !
Moviepy - video ready /content/videos/rl-video-episode-499.mp4


In [18]:
!zip -r /content/videos.zip /content/videos

  adding: content/videos/ (stored 0%)
  adding: content/videos/rl-video-episode-222.mp4 (deflated 5%)
  adding: content/videos/rl-video-episode-55.meta.json (deflated 8%)
  adding: content/videos/rl-video-episode-388.meta.json (deflated 9%)
  adding: content/videos/rl-video-episode-328.meta.json (deflated 8%)
  adding: content/videos/rl-video-episode-207.meta.json (deflated 8%)
  adding: content/videos/rl-video-episode-261.meta.json (deflated 8%)
  adding: content/videos/rl-video-episode-374.meta.json (deflated 8%)
  adding: content/videos/rl-video-episode-163.mp4 (deflated 5%)
  adding: content/videos/rl-video-episode-364.mp4 (deflated 6%)
  adding: content/videos/rl-video-episode-255.meta.json (deflated 8%)
  adding: content/videos/rl-video-episode-447.mp4 (deflated 5%)
  adding: content/videos/rl-video-episode-84.mp4 (deflated 5%)
  adding: content/videos/rl-video-episode-22.mp4 (deflated 5%)
  adding: content/videos/rl-video-episode-252.meta.json (deflated 8%)
  adding: content/vid